In [12]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from keras import layers
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
                '../../data/AugmentedAlzheimerDataset',
                labels="inferred",
                label_mode="int",
                class_names=None,
                color_mode="rgb",
                batch_size=32,
                image_size=(180, 180),  # can change this, maybe avg dimensions over all pics
                shuffle=True,
                seed=69, #haha
                validation_split=.2,
                subset='training',
                interpolation="bilinear",
                follow_links=False,
                crop_to_aspect_ratio=False
            )

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
                '../../data/AugmentedAlzheimerDataset',
                labels="inferred",
                label_mode="int",
                class_names=None,
                color_mode="rgb",
                batch_size=32,
                image_size=(180, 180),  # can change this, maybe avg dimensions over all pics
                shuffle=True,
                seed=69, #haha
                validation_split=.2,
                subset='validation',
                interpolation="bilinear",
                follow_links=False,
                crop_to_aspect_ratio=False
            )

Found 33984 files belonging to 4 classes.
Using 27188 files for training.
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-04-17 23:53:26.445652: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-17 23:53:26.447033: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 33984 files belonging to 4 classes.
Using 6796 files for validation.


### A simple starter to play around with

from: https://www.tensorflow.org/tutorials/images/cnn 

In [22]:
simpleCNN = Sequential()
simpleCNN.add(Conv2D(32, (3,3), activation='relu', input_shape=(180, 180, 3)))
simpleCNN.add(MaxPooling2D((2,2)))
simpleCNN.add(Conv2D(64, (3,3), activation='relu'))
simpleCNN.add(MaxPooling2D((2,2)))
simpleCNN.add(Conv2D(64, (3,3), activation='relu'))

#add dense layers
simpleCNN.add(Flatten())
simpleCNN.add(Dense(64, activation='relu'))
simpleCNN.add(Dense(4))

simpleCNN.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 89, 89, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 43, 43, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 41, 41, 64)        36928     
                                                                 
 flatten_6 (Flatten)         (None, 107584)          

In [25]:
simpleCNN.compile(optimizer='adam', loss = SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = simpleCNN.fit(train_ds, validation_data=val_ds, epochs=5)

Epoch 1/5


2023-04-18 00:29:58.621361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


850/850 [==============================] - ETA: 0s - loss: 39.2653 - accuracy: 0.4695

2023-04-18 00:31:27.019392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


850/850 [==============================] - 97s 113ms/step - loss: 39.2653 - accuracy: 0.4695 - val_loss: 0.8569 - val_accuracy: 0.6149
Epoch 2/5
850/850 [==============================] - 95s 111ms/step - loss: 0.7312 - accuracy: 0.6773 - val_loss: 0.6715 - val_accuracy: 0.7079
Epoch 3/5
850/850 [==============================] - 95s 111ms/step - loss: 0.5190 - accuracy: 0.7757 - val_loss: 0.6834 - val_accuracy: 0.7212
Epoch 4/5
850/850 [==============================] - 95s 111ms/step - loss: 0.3836 - accuracy: 0.8374 - val_loss: 0.6933 - val_accuracy: 0.7450
Epoch 5/5
850/850 [==============================] - 95s 111ms/step - loss: 0.2902 - accuracy: 0.8810 - val_loss: 0.7982 - val_accuracy: 0.7521


## 75% accuracy with boilerplate CNN structure straight from the docs! 
(random would be 25%) 
Let's see how accurate we can get by tweaking the layers/hyperparams to better fit our situation.